In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!mkdir ravdess_data
!unzip 'gdrive/My Drive/Audio_Speech_Actors_01-24.zip' -d ravdess_data

In [ ]:
# !rm -rf ravdess_data/A*/03-01-02*

In [ ]:
#1440
#1248

import pandas as pd
import os
import librosa
import numpy as np
lst = np.array([])
labels = []
i=0

shapes = []
bookmark=0
s_arr = np.zeros((1248,13,216))
for folder in os.listdir('ravdess_data/'):
  for file in os.listdir(f'{"ravdess_data/"}{folder}'):
        X, sample_rate = librosa.load(f'{"ravdess_data/"}{folder}{"/"}{file}', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13)
        
        result = np.zeros((13,216))
        result[:mfccs.shape[0],:mfccs.shape[1]] = mfccs
        s_arr[i] = result
        
        file = int(file[7:8])-1
        if file>0:
          file-=1
        i+=1
        labels.append(file)


In [ ]:
labels = np.array(labels)

In [ ]:
ravdess_X = s_arr
ravdess_y = labels
np.save('ravdess_X.npy',ravdess_X)
np.save('ravdess_y.npy',ravdess_y)

In [ ]:
ravdess_X.shape

(1248, 13, 216, 1)

In [ ]:
from sklearn.utils import shuffle
ravdess_train_X, ravdess_train_y = shuffle(ravdess_train_X, ravdess_train_y, random_state=42)

In [ ]:
from sklearn.utils import shuffle
ravdess_X, ravdess_y = shuffle(ravdess_X, ravdess_y, random_state=42)

In [ ]:
ravdess_X = np.expand_dims(ravdess_X,axis=3)

In [ ]:
from sklearn.model_selection import train_test_split
ravdess_train_X, ravdess_valid_X, ravdess_train_y, ravdess_valid_y  = train_test_split(ravdess_X,ravdess_y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', np.unique(ravdess_train_y),ravdess_train_y)

In [ ]:
!cp ravdess_X.npy 'gdrive/My Drive/AI-Hackathon'
!cp ravdess_y.npy 'gdrive/My Drive/AI-Hackathon'

In [ ]:
!cp 2d_Convolution_Ravdess_Shuffled.h5 'gdrive/My Drive/AI-Hackathon/model'

In [ ]:
ravdess_X.shape

(1248, 13, 216, 1)

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, LSTM, Lambda
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras import backend as K

model_ravdess = Sequential()
kernel = 5
model_ravdess.add(Conv2D(32, 5,strides=2,padding='same',
                 input_shape=(13,216,1)))
model_ravdess.add(Activation('relu'))
model_ravdess.add(BatchNormalization())

# model_ravdess.add(MaxPooling1D(pool_size=(8)))
model_ravdess.add(Conv2D(64, 5,strides=2,padding='same',))
model_ravdess.add(Activation('relu'))
model_ravdess.add(BatchNormalization())

model_ravdess.add(Conv2D(64, 5,strides=2,padding='same',))
model_ravdess.add(Activation('relu'))
model_ravdess.add(BatchNormalization())
# model_ravdess.add(MaxPooling2D(pool_size=(2,3)))
# model_ravdess.add(Lambda(lambda x: K.squeeze(x, axis= 1)))
model_ravdess.add(Flatten())

# model_ravdess.add(LSTM(16))
# model_ravdess.add(Dropout(0.5))

model_ravdess.add(Dense(7))
model_ravdess.add(Activation('softmax'))
# opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)


opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
model_ravdess.summary()

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_133 (Conv2D)          (None, 7, 108, 32)        832       
_________________________________________________________________
activation_158 (Activation)  (None, 7, 108, 32)        0         
_________________________________________________________________
batch_normalization_133 (Bat (None, 7, 108, 32)        128       
_________________________________________________________________
conv2d_134 (Conv2D)          (None, 4, 54, 64)         51264     
_________________________________________________________________
activation_159 (Activation)  (None, 4, 54, 64)         0         
_________________________________________________________________
batch_normalization_134 (Bat (None, 4, 54, 64)         256       
_________________________________________________________________
conv2d_135 (Conv2D)          (None, 2, 27, 64)       

In [ ]:
model_ravdess.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
ravdess_train_X.shape, ravdess_train_y.shape

((998, 13, 216, 1), (998,))

In [ ]:
cnnhistory=model_ravdess.fit(ravdess_X, ravdess_y, batch_size=16, epochs=10, validation_data=None)

Epoch 1/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0453 - acc: 0.9936
Epoch 2/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0305 - acc: 0.9952
Epoch 3/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0201 - acc: 0.9992
Epoch 4/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0123 - acc: 1.0000
Epoch 5/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0092 - acc: 1.0000
Epoch 6/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0091 - acc: 0.9992
Epoch 7/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0075 - acc: 1.0000
Epoch 8/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0050 - acc: 1.0000
Epoch 9/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0048 - acc: 1.0000
Epoch 10/10
1248/1248 [==============================] - 6s 5ms/step - loss: 0.0041 - acc: 1.0000


In [ ]:
model_ravdess.evaluate(ravdess_valid_X,ravdess_valid_y)

250/250 [==============================] - 0s 2ms/step


[1.0253960466384888, 0.6600000028610229]

In [ ]:
model_ravdess.save("lstm_2d_Convolution_Ravdess_Shuffled_new.h5")

In [ ]:
model_ravdes = load_model("2d_Convolution_Ravdess_Shuffled.h5")

In [ ]:
model_ravdes = load_model("lstm_2d_Convolution_Ravdess_Shuffled.h5")

In [ ]:
cp -R 'gdrive/My Drive/Filtered Data' .

In [ ]:

X, sample_rate = librosa.load('neutral_e2.wav', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13)

result = np.zeros((13,216))
result[:mfccs.shape[0],:mfccs.shape[1]] = mfccs
t_arr[0] = result

t_arr.shape

In [ ]:
t_arr = np.zeros((30,13,216))

In [ ]:
!unzip 'gdrive/My Drive/Tata-Training-Diarize.zip'

In [ ]:
# !mv Tata-Training-Diarize/1 Tata-Training-Diarize/temp
# !mv Tata-Training-Diarize/2 Tata-Training-Diarize/1
# !mv Tata-Training-Diarize/temp Tata-Training-Diarize/2

In [ ]:
i=0
test_labels = []
for folder in os.listdir('Tata-Training-Diarize'):
  for file in os.listdir(f'{"Tata-Training-Diarize/"}{folder}'):
    try:
      X, sample_rate = librosa.load(f'{"Tata-Training-Diarize/"}{folder}{"/"}{file}', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
      sample_rate = np.array(sample_rate)
      mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13)

      result = np.zeros((13,216))
      result[:mfccs.shape[0],:mfccs.shape[1]] = mfccs
      t_arr[i] = result

      i+=1
      test_labels.append(int(folder))
    except:
      continue


In [ ]:
t_arr.shape

(30, 13, 216)

In [ ]:
test = np.expand_dims(t_arr,axis=3)

In [ ]:
test.shape

(30, 13, 216, 1)

In [ ]:
classes = ['Neutral','Happy','Sad','Angry','Fearful','Disgust','Surprise']

In [ ]:
preds = model_ravdess.predict_classes(test)
for i in range(len(preds)):
  print(classes[preds[i]], '     ',classes[test_labels[i]])

Sad       Sad
Sad       Sad
Sad       Happy
Sad       Happy
Sad       Happy
Sad       Happy
Sad       Happy
Disgust       Happy
Disgust       Happy
Sad       Fearful
Sad       Angry
Disgust       Angry
Sad       Angry
Sad       Disgust
Sad       Neutral
Disgust       Neutral
Disgust       Neutral
Sad       Neutral
Sad       Neutral
Sad       Neutral
Sad       Neutral
Sad       Neutral
Sad       Neutral
Disgust       Neutral
Sad       Neutral
Sad       Neutral
Happy       Neutral
Sad       Neutral
Happy       Neutral
Sad       Neutral


In [ ]:
ravdess_X = t_arr
ravdess_y = test_labels

In [ ]:
ravdess_X = np.expand_dims(ravdess_X,axis=3)

In [ ]:
from sklearn.model_selection import train_test_split
ravdess_train_X, ravdess_valid_X, ravdess_train_y, ravdess_valid_y  = train_test_split(ravdess_X,ravdess_y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', np.unique(ravdess_train_y),ravdess_train_y)

In [ ]:
cnnhistory=model_ravdess.fit(ravdess_train_X, ravdess_train_y, batch_size=4, epochs=5, validation_data=(ravdess_valid_X, ravdess_valid_y),class_weight=class_weights)

In [ ]:
ravdess_train_y

[0, 0, 3, 2, 1, 0, 1, 5, 3, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1]

In [ ]:
preds = model_ravdess.predict_classes(test)
for i in range(len(preds)):
  print(classes[preds[i]], '     ',classes[test_labels[i]])

In [ ]:
model_ravdess.save('lstm_trained_on_received_data.h5')

In [ ]:
!cp lstm_trained_on_received_data.h5 'gdrive/My Drive/AI-Hackathon/model'

In [ ]:
cp 'gdrive/My Drive/Recording/Angry/Angry.m4a' .

In [ ]:
temp = np.zeros((1,13,216))
X, sample_rate = librosa.load('Angry.m4a', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13)

result = np.zeros((13,216))
result[:mfccs.shape[0],:mfccs.shape[1]] = mfccs
temp[0] = result

temp.shape

(1, 13, 216)

In [ ]:
t = np.expand_dims(temp,axis=3)

In [ ]:
t.shape

(1, 13, 216, 1)

In [ ]:
model_ravdess.predict_classes(t)

array([1])